Author: Wolfgang Black
Date: 2/7/25

## Prose Generation Coding Task

The challenge involves creating a backend system that transforms a series of bulleted "beats" into well-crafted prose that you would find in a novel. This tool is designed to aid writers in fleshing out as scene based on their outlines.

**Part 1: Beats-to-Prose**
Given a set of story beats, like this set for a sci-fi chapter:

1. Begin the chapter with Jack and Xander continuing their excavation on the lunar surface,
creating a sense of tension and anticipation.
2. Describe the barren landscape of the moon, emphasizing its desolation and the isolation
felt by Jack and Xander.
3. Use vivid and descriptive language to portray the moment when Jack and Xander uncover
an alien artifact, highlighting its mysterious and otherworldly appearance.
4. Convey Jack's excitement and trepidation as he realizes the significance of the discovery
and the potential implications it holds for humanity.
5. Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the
urgency of the situation.
6. Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship
and the gravity of the situation.
7. Highlight the importance of the discovery and the potential consequences it could have on
Earth and the lunar mining project.
8. Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a
sense of anticipation and uncertainty.
9. Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges
and dangers that Jack and Xander face.
10. Use sensory details to immerse the reader in the lunar environment, such as the crunch of
lunar soil beneath their boots and the cold, airless atmosphere.
11. Portray Jack's determination and resolve to protect the discovery, even in the face of
potential conflict with rival miners.
12. Hint at the potential dangers and obstacles that Jack and Xander may encounter as they
navigate through a world of corporate greed and betrayal.
13. Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and
anticipation for future chapters.
14. End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to
continue reading and discover what happens next.

Use these beats to produce ~1500 words of prose that matches these beats - in other words, if
a human were to summarize the prose, they would produce something along the lines of these
beats. It's okay to hallucinate details if they are missing from the beats in order to create the
narrative. Target approximately 100-150 words of output per beat. You are allowed to use any LLM or NLP techniques, including open source and closed source services like GPT or Claude.




In [1]:
# Test Input
TEST_BEATS_INPUT = [
    "Begin the chapter with Jack and Xander continuing their excavation on the lunar surface, creating a sense of tension and anticipation.",
    "Describe the barren landscape of the moon, emphasizing its desolation and the isolation felt by Jack and Xander.",
    "Use vivid and descriptive language to portray the moment when Jack and Xander uncover an alien artifact, highlighting its mysterious and otherworldly appearance.",
    "Convey Jack's excitement and trepidation as he realizes the significance of the discovery and the potential implications it holds for humanity.",
    "Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the urgency of the situation.",
    "Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship and the gravity of the situation.",
    "Highlight the importance of the discovery and the potential consequences it could have on Earth and the lunar mining project.",
    "Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a sense of anticipation and uncertainty.",
    "Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges and dangers that Jack and Xander face.",
    "Use sensory details to immerse the reader in the lunar environment, such as the crunch of lunar soil beneath their boots and the cold, airless atmosphere.",
    "Portray Jack's determination and resolve to protect the discovery, even in the face of potential conflict with rival miners.",
    "Hint at the potential dangers and obstacles that Jack and Xander may encounter as they navigate through a world of corporate greed and betrayal.",
    "Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and anticipation for future chapters.",
    "End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to continue reading and discover what happens next."
]

## Solution to part I

To solve this I created a mulitagenic workflow. I decided to utilize LLMs for the majority of this work, using OpenAI's gpt-3.5-turbo for this demo. I've been experimenting with agents and passing data through LLMs recently and wanted to try this instead of more traditional approaches. 

Multi-Agentic Workflow: 
1. We initialize `BeatToStory()` - which is the pipeline class containing our agents.
2. The story beats, provided by the user, are passed into the BeatToStory instance 
3. We run `_.setup_pipe` which initializes the agents for the BeatToStory instance
    - if a list is not provided, we assume a pipeline of `[ContextAgent, ProseAgent, StoryAgent, LengthAgent, FlowAgent]`
4. `ContextAgent` reads through the beats, trying to identify key information, characters, setting, etc
5. `ProseAgent` reads in the beats and the context linking two beats and writes prose to narratively connect the beats while maintaining context
6. `StoryAgent` reviews the context and prose to verify that all the information was maintained and captured by the prose
7. `LengthAgent` reviews the prose length and verifies we're within our `[min, max]` word count per prose
8. `FlowAgent` acts as a final editor, rewriting the overall prose to prevent grammar or repetive language

the `BeatToStory()` class also has methods to disclose cost of each agent (based on shared token costs) and whether each step of the prose was successful. 

Below we can see an example of how well it works. 

I also ran 20 samples, to determine if we're within our word count after the edits done by flow agent. The goal is to be around 1500 works (or 150 words per beat).


In [2]:
# Note: OpenAI init was abstracted into the utils. please set an environment variable OPENAI_KEY to be accessible by `os.environ.get('OPENAI_KEY')`
import sys
sys.path.append('..') #only needed in notebook
from utils import BeatToStory
beatbot = BeatToStory()

In [3]:
# beatbot.beats = TEST_BEATS_INPUT
# beatbot.setup_pipeline()
# print(beatbot.describe_pipeline())

In [4]:
# beatbot.pipe()
# print(len(beatbot.story.split()))
# print(beatbot.story_length)
# print(beatbot.pipeline_cost())
# print(beatbot.edited_story)

In [5]:
# for i in range(20):
#     beatbot.pipe()
#     print(f"beatbot story gen {i}")
#     print(f"length of unedited story: {len(beatbot.story.split())}")
#     print(f"length of edited story: {beatbot.story_length}")
#     print('-'*50+'\n')

### **Part 2: Incorporating Story Metadata**

Now we want to extend the functionality of Beats-to-Prose to accept a few new parameters:
- a list of characters (name and details about them)
- setting
- genre
- style of prose and have that alter the generated prose.

Describe in detail your approach to this. Some things to consider:

1. What models would you use? Why?
2. If you need to procure data, how would go about doing that? How would you prepare the data?
3. What is your approach to producing prose that matches the parameters?

How would you evaluate whether the generations are good quality and fit the parameters specified?
Consider cost and latency considerations

## Solution to Part II


In [ ]:
from utils import ContextAgent, MetadataAgent, ProseAgent, StyleGenreAgent, StoryAgent, LengthAgent, FlowAgent

metabot = BeatToStory()
metabot.beats = TEST_BEATS_INPUT

metabot.user_metadata = {"setting":
        {"location": "lunar surface",
         "notes": "The Jack and Xander are alone. Dr. Thorne is in her lab. They are on the dark side of the moon",
        },
    "genre": "romance",
    "style": "voltair - no punctuation", 
    "characters": [
        {"name": "Jack", "profile": "An astronaut miner working on the moon. An educated adventure, he's working alongside Xander and Dr. Thorne searching the moon for precious metals. He's currently contracted to the Corp (a big pharma corp), but has a distrust of large companies. His main goal is to prevent the privatization of data that could help humanity. He's our main character loosely modeled off of Indian Jones"},
        {"name": "Xander", "profile": "An astronaught miner working alongside Jack on the moon. Xander served in the British Royal Space Force and bounced around from career to career. He's got serious debts and a doubious history... Is he really trust worthy? He met Jack during one of Jacks expidition to the Sahara, where Jack saved his life. He's worked along side Jack any time Jacks needed support flying - whether that's terrestrial or beyond"},
        {"name": "Dr. Selene Thorne", "Profile": "Selene Thorne is a brilliant Astrobiophysics. During her graduate program she discovered frequencies from deepspace that seemed to be bouncing off the moon. These frequencies seemingly contained the genetic code of the human race - and were set on repeat. Her lifes work has been to find some sign of those frequecies, how they're being broadcast, and from where. She's worked closely with Jack behind the scenes, trying to prevent evil pharam corps from privatizing the data and charging for the amazing potential medical advances. She has some private shared history with Jack - but the only sign of it in their interactions is deep trust."}]}

metabot.agents = {
                "context": ContextAgent(),
                "meta": MetadataAgent(),
                "prose": ProseAgent(min_words=metabot.min_words_per_beat, 
                             max_words=metabot.max_words_per_beat),
                "genre": StyleGenreAgent(style_guide = metabot.genre),
                "style": StyleGenreAgent(style_guide = metabot.style),
                "story": StoryAgent(),
                "length": LengthAgent(min_words=metabot.min_words_per_beat, 
                                    max_words=metabot.max_words_per_beat),
                "flow": FlowAgent()
            }

metabot.pipe()
print(len(metabot.story.split()))
print(metabot.story_length)
print(metabot.pipeline_cost())
print(metabot.edited_story)

## **Part 3: Describe how you might turn this code into an API**

Imagine you have a working protoype and would like the application team to start using your solution. You can assume you are using the FastAPI framework or something similar. You can use psuedocode for this exercise.

1. What would a hander look like, and what parameters would it receive?

In [9]:
# Write your solution for the handler here You don't have to run this cell
from fastapi import FastAPI
from fastapi.routing import APIRoute

app = FastAPI()

@app.post("")
async def beat_to_prose_generation():
    return []
